In [41]:
import os

In [42]:
%pwd

'c:\\Users\\Hp'

In [43]:
os.chdir("c:/Users/Hp/Desktop/Endtoendml")  # Adjust the path to your project root

In [44]:
print(os.getcwd())

c:\Users\Hp\Desktop\Endtoendml


In [45]:
# Data Validation Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [46]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories

In [55]:
# update the configuration manager in src config

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        all_schema = self.schema["columns"]

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir = Path(config.root_dir),
            STATUS_FILE = config.STATUS_FILE,
            unzip_data_dir = Path(config.unzip_data_dir),
            all_schema = all_schema,
        )

        return data_validation_config

In [56]:
import os
from mlProject import logger
import pandas as pd

In [57]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self)-> bool:
        try:
            validation_status = None

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)

            all_schema = self.config.all_schema.keys()


            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

                else:
                    validation_status = True
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")

                    return validation_status
                
        except Exception as e:
            raise e

In [58]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    data_validation.validate_all_columns()
except Exception as e:
    raise e

[2025-02-28 21:09:57,589: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-28 21:09:57,593: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-28 21:09:57,611: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-28 21:09:57,614: INFO: common: created directory at: artifacts]
[2025-02-28 21:09:57,616: INFO: common: created directory at: artifacts/data_validation]
